In [1]:
import pandas as pd
df=pd.read_csv('new_data.csv')
df=df[(df.type_char!='file_folder')&(df.type_char!='handwritten')]
df.type.replace(list(df.type.unique()),[i for i in range(21)],inplace=True)
df.head()

,Unnamed: 0,filename,type,text_image,type_char
0,0,img_0000000.jpg,0,FACTURE\n\nLOGO\n\nJoanna Binet\n48 Coubertin\...,facture
1,1,img_0000001.jpg,0,Joanna Binet\n48 Coubertin\n31400 Paris\n\nFAC...,facture
2,2,img_0000002.jpg,0,FACTURE\n\nMon entreprise : Nom de la société\...,facture
3,3,img_0000003.jpg,0,Joanna Binet\n48 Coubertin\n31400 Paris\n\nFAC...,facture
4,4,img_0000004.jpg,0,Payer en ligne >\nFACTURE No\n\nSFIDELI\n\nF/0...,facture


In [9]:
#nettoyage de text 1

#from nltk.corpus import stopwords

#stop_words = set(stopwords.words('french'))
#stop_words.update(["." , ",",':',"[{'translation_text':"])

#def stop_words_filtering(liste):
    #liste_new=[]
    #for i in liste:
        #if i not in stop_words:
            #liste_new.append(i)
        #else:
            #continue 
    #return liste_new
#df['text_image_fr']=[(str(' '.join(stop_words_filtering(txt.split())))[:-2][1:-1])  for txt in df.text_image_fr]

In [3]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test=train_test_split(df.text_image.astype(str).values.tolist(),df.type.values.tolist(),test_size=0.2)

In [4]:
from transformers import BertTokenizerFast, BertForSequenceClassification

model = "bert-base-uncased"
tokenizer = BertTokenizerFast.from_pretrained(model, do_lower_case=True)

In [5]:
max_length = 512

X_train_encoded = tokenizer(X_train, truncation=True, padding=True, max_length=max_length)
X_test_encoded = tokenizer(X_test, truncation=True,padding=True, max_length=max_length)

In [6]:
import torch


class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)
    
    
train_dataset = NewsGroupsDataset(X_train_encoded, y_train)
test_dataset = NewsGroupsDataset(X_test_encoded, y_test)

In [7]:
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained(
    model, num_labels=len(df.type.unique()))
training_args = TrainingArguments(
    output_dir='./results',          # chemin de stockage des résultats
    num_train_epochs=10,              # nombre d'époques pour l'entraînement
    per_device_train_batch_size=8,  # batch size pour l'entraînement
    per_device_eval_batch_size=20,   # batch size pour l'évaluation du modèle
    warmup_steps=500,                # nombre d'étapes pour le pas d'apprentissage
    weight_decay=0.01,               # paramètre décidant des poids
    logging_dir='./logs',            # chemin de stockage des logs
    # utilisation du meilleur modèle à l'issu de l'entraînement
    load_best_model_at_end=True,
    logging_steps=400,               # log & enregistrer les poids à chaque étape
    save_steps=400,
    evaluation_strategy="steps",     # évaluation à chaque `logging_steps`
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)
trainer.train()
trainer.evaluate()

***** Running training *****
  Num examples = 1235
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 465


Step,Training Loss,Validation Loss
400,2.086800,1.248358


***** Running Evaluation *****
  Num examples = 309
  Batch size = 20
Saving model checkpoint to ./results\checkpoint-400
Configuration saved in ./results\checkpoint-400\config.json
Model weights saved in ./results\checkpoint-400\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./results\checkpoint-400 (score: 1.248358130455017).
***** Running Evaluation *****
  Num examples = 309
  Batch size = 20


{'eval_loss': 1.248358130455017,
 'eval_runtime': 369.8996,
 'eval_samples_per_second': 0.835,
 'eval_steps_per_second': 0.043,
 'epoch': 3.0}

In [11]:
model.save_pretrained("Bert_nlp")

Configuration saved in Bert_nlp\config.json
Model weights saved in Bert_nlp\pytorch_model.bin


In [12]:
import numpy as np
y_pred = trainer.predict(test_dataset)
y_pred = np.argmax(y_pred.predictions, axis=-1)

***** Running Prediction *****
  Num examples = 309
  Batch size = 20


In [13]:
from sklearn.metrics import classification_report

print( classification_report(y_test, y_pred) )

              precision    recall  f1-score   support

           0       0.55      1.00      0.71         6
           1       0.89      0.78      0.83        32
           2       0.00      0.00      0.00        10
           3       0.93      0.45      0.61        31
           4       0.88      0.70      0.78        50
           5       0.45      0.93      0.60        41
           6       0.72      0.97      0.83        62
           7       0.00      0.00      0.00         3
           8       0.30      1.00      0.46         3
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00         6
          11       0.17      1.00      0.29         1
          12       0.80      1.00      0.89         4
          13       0.00      0.00      0.00        10
          14       0.00      0.00      0.00         5
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00         6
          17       0.00    

C:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
